In [ ]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import logging
import torch
import math
import time
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler, convert_tokens_to_words
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict

from transformers import BertTokenizer,BertModel,BertTokenizerFast
from utils.MIND import MIND

logger = logging.getLogger(__name__)

In [ ]:
t = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.signal_length = 10
# config.scale = 'large'
# config.impr_size = 100
# config.mode = 'test'

path = config.path + 'MIND/MINDdemo_train/'
a = MIND(config, path + 'news.tsv', path + 'behaviors.tsv')

In [29]:
a.attn_mask[2].sum(), a.encoded_news[2], a.attn_mask_dedup[2], a.attn_mask_dedup[2].sum()

(32,
 array([  101,  2753,  5409, 14243,  2005,  7579,  6638,  2122,  9428,
        19741, 14243,  2024,  3173,  2017,  2067,  1998,  4363,  2017,
         2013,  8328,  4667,  2008, 18162,  7579,  6638,  2005,  2204,
         1012,  2740,  3635, 10483,  2015,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
a.subwords[3], t.convert_ids_to_tokens(a.encoded_news[3][:32]), a.attn_mask[3].sum()

In [ ]:
a.attn_mask_dedup

In [ ]:
a[1]['his_attn_mask']

In [ ]:
r = np.array([[[1,1],[2,2],[2,3]],[[1,1],[1,2],[1,3]]])
r[r[:,:,0] == 1]

In [27]:
t.convert_ids_to_tokens(a.encoded_news[3])

['[CLS]',
 'the',
 'cost',
 'of',
 'trump',
 "'",
 's',
 'aid',
 'freeze',
 'in',
 'the',
 'trenches',
 'of',
 'ukraine',
 "'",
 's',
 'war',
 'lt',
 '.',
 'ivan',
 'mo',
 '##lch',
 '##ane',
 '##ts',
 'peeked',
 'over',
 'a',
 'parapet',
 'of',
 'sand',
 'bags',
 'at',
 'the',
 'front',
 'line',
 'of',
 'the',
 'war',
 'in',
 'ukraine',
 '.',
 'next',
 'to',
 'him',
 'was',
 'an',
 'empty',
 'helmet',
 'propped',
 'up',
 'to',
 'trick',
 'sniper',
 '##s',
 ',',
 'already',
 'per',
 '##for',
 '##ated',
 'with',
 'multiple',
 'holes',
 '.',
 'news',
 'news',
 '##world',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [ ]:
loader1 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, sampler=Partition_Sampler(a,2,0))
records1 = list(loader1

In [ ]:
records1[0]['his_attn_mask']

In [ ]:
dic = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDdemo_train/news.pkl', 'rb'))

In [ ]:
sb = dic['subwords_first']
sb

In [ ]:
np.unique(a[:,0]),a

In [ ]:
t.tokenize("I don't give a fuck")